<a href="https://colab.research.google.com/github/ChitralaDhruv/-X/blob/main/100bps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import csv

# Load your data from an Excel file (adjust the file name as needed)
file_path = '/content/Silver_30Min.xlsx'  # Replace with your file path
data = pd.read_excel(file_path)

# Drop rows with missing values in the required columns (if any)
data.dropna(subset=['Local Date', 'Local Time', 'Close'], inplace=True)

# Sort data by 'Local Date' and 'Local Time' (optional if the data isn't sorted)
data.sort_values(by=['Local Date', 'Local Time'], inplace=True)

# Strategy parameters
initial_price = 22  # Starting price
price_step = 0.01      # 1% steps for buy orders
lots = 0
positions = []         # Track all open positions and their take-profits
profits = 0            # Track total profits from successful take-profits

# List to store trade data for CSV output
trade_log = []

# Function to place a buy and corresponding take-profit order
def place_order(entry_price, timestamp):
    global lots
    take_profit_price = entry_price * (1 + price_step)  # 1% above entry price
    positions.append({'entry_price': entry_price, 'take_profit': take_profit_price, 'timestamp': timestamp})
    lots += 1
    print(f"Buy Order: Bought at {entry_price} on {timestamp}, Take-Profit set at {take_profit_price}")

# Initialize with the first buy
initial_timestamp = f"{data.iloc[0]['Local Date']} {data.iloc[0]['Local Time']}"
place_order(initial_price, initial_timestamp)

# Loop through the historical data
for index, row in data.iterrows():
    current_price = row['Close']  # Use the 'Close' price for each 30-min candle
    timestamp = f"{row['Local Date']} {row['Local Time']}"

    # Check if any take-profit orders are hit
    for pos in positions.copy():
        if current_price >= pos['take_profit']:
            # Take profit hit, record profit and remove position
            profit = pos['take_profit'] - pos['entry_price']
            profits += profit
            positions.remove(pos)
            print(f"Take-Profit Hit: Sold at {pos['take_profit']} on {timestamp}, Profit: {profit}")
            # Log the trade in the trade_log list for CSV export
            trade_log.append({
                'Entry Price': pos['entry_price'],
                'Entry Date Time': pos['timestamp'],
                'Exit Price': pos['take_profit'],
                'Exit Date Time': timestamp,
                'Profit': profit
            })
            # Re-enter the market by placing another buy at the same price
            place_order(pos['entry_price'], timestamp)

    # Check if price has dropped 1% below the last entry price and place a new buy order
    last_position = positions[-1]  # Last buy price
    if current_price <= last_position['entry_price'] * (1 - price_step):
        new_entry_price = last_position['entry_price'] * (1 - price_step)
        place_order(new_entry_price, timestamp)

# At the end of the backtest, calculate unrealized loss for remaining positions
unrealized_loss = 0
current_price = data.iloc[-1]['Close']  # Final close price
for pos in positions:
    unrealized_loss += (pos['entry_price'] - current_price)

# Save the trade log to a CSV file
output_file = 'trade_log.csv'
trade_log_df = pd.DataFrame(trade_log)
trade_log_df.to_csv(output_file, index=False)

# Output final results
print(f"\nBacktest Summary:")
print(f"Total Profit: {profits}")
print(f"Unrealized Loss: {unrealized_loss}")
print(f"Number of Lots Held: {lots}")
print(f"Trade log saved to {output_file}")


Buy Order: Bought at 22 on 2023-10-20 00:30:00 2023-10-20 00:30:00, Take-Profit set at 22.22
Take-Profit Hit: Sold at 22.22 on 2023-10-20 00:30:00 2023-10-20 00:30:00, Profit: 0.21999999999999886
Buy Order: Bought at 22 on 2023-10-20 00:30:00 2023-10-20 00:30:00, Take-Profit set at 22.22
Take-Profit Hit: Sold at 22.22 on 2023-10-20 01:00:00 2023-10-20 01:00:00, Profit: 0.21999999999999886
Buy Order: Bought at 22 on 2023-10-20 01:00:00 2023-10-20 01:00:00, Take-Profit set at 22.22
Take-Profit Hit: Sold at 22.22 on 2023-10-20 01:30:00 2023-10-20 01:30:00, Profit: 0.21999999999999886
Buy Order: Bought at 22 on 2023-10-20 01:30:00 2023-10-20 01:30:00, Take-Profit set at 22.22
Take-Profit Hit: Sold at 22.22 on 2023-10-20 02:00:00 2023-10-20 02:00:00, Profit: 0.21999999999999886
Buy Order: Bought at 22 on 2023-10-20 02:00:00 2023-10-20 02:00:00, Take-Profit set at 22.22
Take-Profit Hit: Sold at 22.22 on 2023-10-20 02:30:00 2023-10-20 02:30:00, Profit: 0.21999999999999886
Buy Order: Bought at

In [3]:
import pandas as pd
import csv

maxi = 0
# Load your data from an Excel file (adjust the file name as needed)
file_path = '/content/MOODENG-5m.xlsx'  # Replace with your file path
data = pd.read_excel(file_path)

# Drop rows with missing values in the required columns (if any)
data.dropna(subset=['Local Date', 'Close'], inplace=True)

# Sort data by 'Local Date' and 'Local Time' (optional if the data isn't sorted)
data.sort_values(by=['Local Date',], inplace=True)

# Strategy parameters
SH = 0.8  # Starting price
price_step = 0.05      # 1% steps for buy orders
lots = 0
positions = []         # Track all open positions and their take-profits
profits = 0            # Track total profits from successful take-profits

# List to store trade data for CSV output
trade_log = []



# Function to place a buy and corresponding take-profit order
def place_order(entry_price, timestamp):
    global lots, maxi
    take_profit_price = round(entry_price * (1 + price_step), 5)  # 1% above entry price
    positions.append({'entry_price': entry_price, 'take_profit': take_profit_price, 'timestamp': timestamp})
    lots += 1
    print(f"Buy Order: Bought at {entry_price} on {timestamp}, Take-Profit set at {take_profit_price}")
    maxi = max(maxi, len(positions))

# Initialize with the first buy
initial_timestamp = f"{data.iloc[0]['Local Date']}"

initial_price = data.iloc[0]['Close']
x = SH
while(initial_price < x):
  place_order(x, initial_timestamp)
  x = round(x*(1-price_step), 5)

# Loop through the historical data
for index, row in data.iterrows():
    open = row['Open']
    high = row['High']
    low = row['Low']
    close = row['Close']

    timestamp = f"{row['Local Date']}"

    if(high > positions[-1]['take_profit']):
      for pos in positions.copy():
        if high >= pos['take_profit']:
            # Take profit hit, record profit and remove position
            profit = pos['take_profit'] - pos['entry_price']
            profits += profit
            positions.remove(pos)
            print(f"Take-Profit Hit: Sold at {pos['take_profit']} on {timestamp}, Profit: {profit}")
            # Log the trade in the trade_log list for CSV export
            trade_log.append({
                'Entry Price': pos['entry_price'],
                'Entry Date Time': pos['timestamp'],
                'Exit Price': pos['take_profit'],
                'Exit Date Time': timestamp,
                'Profit': profit
            })
    else:
      new = round(positions[-1]['entry_price']*(1-price_step), 5)
      while(low < new):
        place_order(new, initial_price)
        new = round(positions[-1]['entry_price']*(1-price_step), 5)


unrealized_loss = 0
current_price = data.iloc[-1]['Close']  # Final close price
for pos in positions:
    unrealized_loss += (pos['entry_price'] - current_price)

# Save the trade log to a CSV file
output_file = 'trade_log.csv'
trade_log_df = pd.DataFrame(trade_log)
trade_log_df.to_csv(output_file, index=False)

# Output final results
print(f"\nBacktest Summary:")
print(f"Total Profit: {profits}")
print(f"Unrealized Loss: {unrealized_loss}")
print(f"Number of Lots Held: {lots}")
print(f"Trade log saved to {output_file}")

print(maxi)







Buy Order: Bought at 0.8 on 1730390400000.0, Take-Profit set at 0.84
Buy Order: Bought at 0.76 on 1730390400000.0, Take-Profit set at 0.798
Buy Order: Bought at 0.722 on 1730390400000.0, Take-Profit set at 0.7581
Buy Order: Bought at 0.6859 on 1730390400000.0, Take-Profit set at 0.7202
Buy Order: Bought at 0.6516 on 1730390400000.0, Take-Profit set at 0.68418
Buy Order: Bought at 0.61902 on 1730390400000.0, Take-Profit set at 0.64997
Buy Order: Bought at 0.58807 on 1730390400000.0, Take-Profit set at 0.61747
Buy Order: Bought at 0.55867 on 1730390400000.0, Take-Profit set at 0.5866
Buy Order: Bought at 0.53074 on 1730390400000.0, Take-Profit set at 0.55728
Buy Order: Bought at 0.5042 on 1730390400000.0, Take-Profit set at 0.52941
Buy Order: Bought at 0.47899 on 1730390400000.0, Take-Profit set at 0.50294
Buy Order: Bought at 0.45504 on 1730390400000.0, Take-Profit set at 0.47779
Buy Order: Bought at 0.43229 on 1730390400000.0, Take-Profit set at 0.4539
Buy Order: Bought at 0.41068 on 1